In [29]:
import json
import os
from PIL import Image

def convert_bbox(size, box):
    dw = 1.0 / size[0]
    dh = 1.0 / size[1]
    x_min, y_min, width, height = box
    x_center = (x_min + (width / 2)) * dw
    y_center = (y_min + (height / 2)) * dh
    width_normalized = width * dw
    height_normalized = height * dh
    return (x_center, y_center, width_normalized, height_normalized)

def convert_coco_to_yolo(json_file, image_dir, output_dir, original_size=(1024, 1024)):
    try:
        with open(json_file, 'r') as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"Error: JSON file {json_file} not found.")
        return
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in {json_file}")
        return

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for image_data in data['images']:
        image_id = image_data['id']
        file_name = image_data['file_name']
        img_path = os.path.join(image_dir, file_name)

        if not os.path.exists(img_path):
            print(f"Warning: Image {file_name} not found in {image_dir}")
            continue

        try:
            img = Image.open(img_path)
            width, height = img.size
        except IOError:
            print(f"Error: Unable to open image {file_name}")
            continue

        yolo_bbox_list = []
        for ann in data['annotations']:
            if ann['image_id'] == image_id:
                bbox = ann['bbox']  # [x_min, y_min, width, height]
                category_id = ann['category_id']
                
                # YOLO 포맷으로 변환
                yolo_bbox = convert_bbox(original_size, bbox)

                # YOLO 모델의 입력 크기에 맞게 조정 (정규화된 값 사용)
                yolo_bbox_list.append(f"{category_id} {' '.join([f'{a:.6f}' for a in yolo_bbox])}")

        # Write YOLO format file
        txt_name = os.path.splitext(os.path.basename(file_name))[0] + '.txt'
        txt_path = os.path.join(output_dir, txt_name)
        os.makedirs(os.path.dirname(txt_path), exist_ok=True)
        with open(txt_path, 'w') as f:
            f.write('\n'.join(yolo_bbox_list))

    print(f"Conversion completed. Labels saved in {output_dir}")

# 절대 경로를 사용하여 함수 호출
json_file = '/data/ephemeral/home/dataset/train.json'
image_dir = '/data/ephemeral/home/dataset/'
output_dir = '/data/ephemeral/home/dataset/labels'

convert_coco_to_yolo(json_file, image_dir, output_dir, original_size=(1024, 1024))


Conversion completed. Labels saved in /data/ephemeral/home/dataset/labels
